<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/5_KoBERT_dial_eval_ks_kw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_ks.pt'
TESTSET_PATH = '/content/drive/MyDrive/KAIST/SEP531/kw_10000.csv'

In [2]:
!ls -l /content/drive/MyDrive/KAIST/SEP531

ls: cannot access '/content/drive/MyDrive/KAIST/SEP531': No such file or directory


#KoBERT 다운로드(사용)#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-mgno6ij9
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-mgno6ij9
     |████████████████████████████████| 344 kB 4.3 MB/s 
     |████████████████████████████████| 46.9 MB 59.1 MB/s 
     |████████████████████████████████| 4.9 MB 40.6 MB/s 
     |████████████████████████████████| 1.2 MB 38.4 MB/s 
     |████████████████████████████████| 3.3 MB 40.6 MB/s 
     |████████████████████████████████| 3.3 MB 35.1 MB/s 
     |████████████████████████████████| 61 kB 548 kB/s 
     |████████████████████████████████| 596 kB 35.9 MB/s 
     |████████████████████████████████| 895 kB 51.7 MB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=fba3251ff5208a664b69210ff4648e7a30ccf099fdb146e9cb2fc4b82de820e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-jgljuk7w/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddf

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#실행 환경(사용)#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [8]:
!pip install -U torchtext==0.6.0
!pip install transformers

     |████████████████████████████████| 64 kB 1.8 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.11.0
    Uninstalling torchtext-0.11.0:
      Successfully uninstalled torchtext-0.11.0


In [9]:
#!pip install mxnet
#!pip install gluonnlp pandas tqdm
#!pip install sentencepiece
#!pip install transformers==3.0.2
#!pip install torch

# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch

# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [10]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#KoBERT 입력 데이터로 변환(사용)#

In [11]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [12]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#추출한 데이터에 감정 레이블 추가(사용)#

In [13]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

새로운 문장을 테스트 할 때, 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성해주어야 한다. 아래 코드를 작성하여 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들어 주었다.

In [14]:
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
model1.load_state_dict(torch.load(SAVEPOINT_PATH), strict=False)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [15]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

dic = {0:'공포', 1:'놀람', 2:'분노', 3:'슬픔', 4:'중립', 5:'행복', 6:'혐오'}
dic = {0:'fear', 1:'surprise', 2:'anger', 3:'sadness', 4:'neutral', 5:'happiness', 6:'disgust'}
r_dic = {'fear':0, 'surprise':1, 'anger':2, 'sadness':3, 'neutral':4, 'happiness':5, 'disgust':6}
def getEmotion(out):
  idx_eval=[]
  val_eval=[]
  for i in out:
    logits=i
    logits = logits.detach().cpu().numpy()
    idx = np.argmax(logits)
    idx_eval.append(idx)
    val_eval.append(dic.get(idx))
    return idx_eval[0], val_eval[0]


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        return getEmotion(out)


        # file wrting

using cached model


#Eval

In [16]:
import csv

texts = []
labels = []

with open(TESTSET_PATH, 'r') as f:
  csv_data = csv.reader(f, delimiter='|')
  next(csv_data)
  for i, line in enumerate(csv_data):
    text = line[0]
    label = int(line[1])
    texts.append(text)
    labels.append(label)

In [17]:
len(texts)

9999

In [18]:
y = []
p = []
def Eval():
  for i in range(len(texts)):
    text = texts[i]
    label = labels[i]
    label_idx = label
    #label_idx = r_dic.get(label)
    idx, val = predict(text)
    #print('(%s/%s) %s\n' % (val, label, text))
    if(i % 1000 == 0):
      print('[%d](%c)(%d/%d) %s\n' % (i, 'O' if (idx == label_idx) else 'X', idx, label_idx, text))
    p.append(idx)
    y.append(label_idx)

  #print('>> Generated file %s/%s.answer\n' % (path, jsonfilename))

  #with open('%s/%s.data' % (path, jsonfilename), 'w') as f:
  #  for u in ut:
  #    f.write('%s\t%s\n' % (u['standard_form'], u['dialect_form']))


In [19]:
Eval()

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[0](X)(0/6) 이 날라리야 워드메서 나쁜 것만 닮아가지구.

[1000](X)(6/4) 낼 실망시키지 마세요.

[2000](X)(3/0) 그런데 잠이 들어가는 애를 툭 근디믄 엄마한테 혼나기도 했는데.

[3000](O)(4/4) 상구 계획이 서지 않았어.

[4000](O)(4/4) 실실 집에 가고 싶은 모양이구나.

[5000](O)(0/0) 지굼 통화 중인 전화를 끊고 이 전화를 받으시겠어요?

[6000](O)(4/4) 머이 좋은지 보세요.

[7000](O)(0/0) 그근 왜 물으시는 건가요.

[8000](O)(4/4) 그는 우물거리며 먼 말을 했지만 나는 알아들을 수가 없었어.

[9000](O)(4/4) 그 나무는 땅거무 속에서 어렴풋이 거하게 보였어.



#F1 score

In [20]:
import sklearn.metrics as metrics
from sklearn.metrics import roc_auc_score

# sklearn 을 이용하면 전부 계산해준다.
print('accuracy', metrics.accuracy_score(y,p) )
print('precision1', metrics.precision_score(y, p, average=None)) #
print('precision2', metrics.precision_score(y, p, average=None).mean()) #
print('precision3', metrics.precision_score(y, p, average='macro')) #1.0
print('precision4', metrics.precision_score(y, p, average='micro')) #1.0
print('recall1', metrics.recall_score(y, p, average=None)) #
print('recall2', metrics.recall_score(y, p, average=None).mean()) #
print('recall3', metrics.recall_score(y, p, average='macro')) #1.0
print('recall4', metrics.recall_score(y, p, average='micro')) #1.0
print('f1_score1', metrics.f1_score(y, p, average=None)) #
print('f1_score2', metrics.f1_score(y, p, average=None).mean()) #
print('f1_score3', metrics.f1_score(y, p, average='macro')) #1.0
print('f1_score4', metrics.f1_score(y, p, average='micro')) #1.0

print(metrics.classification_report(y,p))
print(metrics.confusion_matrix(y,p))

accuracy 0.6687668766876688
precision1 [0.64861111 0.62349676 0.55894309 0.70630631 0.71866295 0.73926868
 0.3704698 ]
precision2 0.6236798144077985
precision3 0.6236798144077985
precision4 0.6687668766876688
recall1 [0.51094092 0.58304498 0.46768707 0.63191832 0.7964854  0.65677966
 0.49197861]
recall2 0.5912621380119611
recall3 0.5912621380119611
recall4 0.6687668766876688
f1_score1 [0.57160343 0.60259276 0.50925926 0.66704481 0.75557558 0.69558714
 0.42266462]
f1_score2 0.603475370692138
f1_score3 0.603475370692138
f1_score4 0.6687668766876688
              precision    recall  f1-score   support

           0       0.65      0.51      0.57       914
           1       0.62      0.58      0.60      1156
           2       0.56      0.47      0.51       588
           3       0.71      0.63      0.67      1861
           4       0.72      0.80      0.76      4211
           5       0.74      0.66      0.70       708
           6       0.37      0.49      0.42       561

    accuracy 

In [21]:
from sklearn.metrics import roc_auc_score

def roc_auc_score_multiclass(actual_class, pred_class, average = "macro"):

  #creating a set of all the unique classes using the actual class list
  unique_class = set(actual_class)
  roc_auc_dict = {}
  for per_class in unique_class:
    #creating a list of all the classes except the current class 
    other_class = [x for x in unique_class if x != per_class]

    #marking the current class as 1 and all other classes as 0
    new_actual_class = [0 if x in other_class else 1 for x in actual_class]
    new_pred_class = [0 if x in other_class else 1 for x in pred_class]

    #using the sklearn metrics method to calculate the roc_auc_score
    roc_auc = roc_auc_score(new_actual_class, new_pred_class, average = average)
    roc_auc_dict[per_class] = roc_auc

  return roc_auc_dict

print("\nLogistic Regression")
# assuming your already have a list of actual_class and predicted_class from the logistic regression classifier
lr_roc_auc_multiclass = roc_auc_score_multiclass(y, p)
print(lr_roc_auc_multiclass)


Logistic Regression
{0: 0.741546408885688, 1: 0.7685099390482258, 2: 0.7223144756787, 3: 0.7859149248277529, 4: 0.7848183715043876, 5: 0.8195640851635171, 6: 0.7211429390573776}
